In [1]:
!pip install datasets -q
# !pip install bm25s[full] -q

# # If you want to use stemming for better results, you can install a stemmer
# !pip install PyStemmer -q

# # To speed up the top-k selection process, you can install `jax`
# !pip install jax[cpu] -q
# !pip install lightning -q
# !pip install hydra-core --upgrade -q
!pip install -U datasets -q
!pip install fsspec==2023.6.0 -q
# !pip install memory-profiler -q
!pip install safetensors>=0.4.3


In [2]:
!pip install matplotlib -q


# Data

## Processing CryptOQA-2024 data

In [32]:
from datasets import load_dataset

train_qna_crypto_data = {
    "train": "QnA-TRAIN.csv"
}

train_qna_data = load_dataset("itwastony/CryptOQA", data_files=train_qna_crypto_data)


QnA-TRAIN.csv:   0%|          | 0.00/40.3M [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [40]:
train_qna_data_df = train_qna_data["train"].to_pandas()


In [43]:
qna_relevant = train_qna_data_df[train_qna_data_df.relevance == 1]


In [58]:
qna_relevant["comment_body_len"] = [len(text) for text in qna_relevant.comment_body.tolist()]


/tmp/ipykernel_882765/1641044695.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  qna_relevant["comment_body_len"] = [len(text) for text in qna_relevant.comment_body.tolist()]


In [83]:
max_comment_len = 200

filtered_data_qna = qna_relevant[(qna_relevant.comment_body_len >= max_comment_len)
            & (qna_relevant.selftext != "[removed]")]\
    .dropna().copy()


In [88]:
print(filtered_data_qna.MAIN.iloc[1])


looking for assistance with doge total noob here. i was gifted some dogecoin shortly after it came out and just found my old wallet on an old hard drive. i downloaded the new version of dogecoin core and followed a guide on how to get the address for the transaction. i just have no clue how to move this from my wallet to a trading app like robinhood. is that even possible? just trying to figure out how to sell it if i wanted to.


In [89]:
print(filtered_data_qna.comment_body.iloc[1])


First of all Congratulations,\n\nSecond of all I would recommend that you don't use Robinhood.\n\nTry moving your doge to a better reputable exchange that supports it , like binance and trade to any other coin or fiat pairing of your liking.


In [92]:
data_columns = [
    "MAIN",
    "comment_body"
]

crypto_qna_benchmark = filtered_data_qna[data_columns].copy()


In [94]:
crypto_qna_benchmark.to_csv(
    "/share/a.tikhonov/datasets/other_exps/qna_filterd_benchmark.csv",
    index=False
)


## CryptoQA data (*HemaChandrao/crypto_QA*)

In [6]:
from datasets import load_dataset

crypto_qa_hf_benchmark = load_dataset("HemaChandrao/crypto_QA")


In [7]:
queries = [crypto_qa_hf_benchmark[part]["conversations"][idx][0]["content"] 
           for part in ["train", "validation", "test"]
           for idx in range(len(crypto_qa_hf_benchmark[part]["conversations"]))]

answers = [crypto_qa_hf_benchmark[part]["conversations"][idx][1]["content"] 
           for part in ["train", "validation", "test"]
           for idx in range(len(crypto_qa_hf_benchmark[part]["conversations"]))]


In [8]:
import pandas as pd

retrieval_bench = pd.DataFrame(
    {
        "queries": queries,
        "answers": answers
    }
)


In [10]:
retrieval_bench.to_csv(
    "/share/a.tikhonov/datasets/other_exps/rag_retrieve_bench.csv",
    index=False
)


## Data markup with Qwen

In [4]:
!pip install -U sentence-transformers -q
!pip install -U transformers -q
!pip install -U accelerate -q


In [16]:
import pandas as pd

crypto_qna_benchmark = pd.read_csv(
    "/share/a.tikhonov/datasets/other_exps/qna_filtered_benchmark.csv"
)
crypto_retrieve_bench = pd.read_csv(
    "/share/a.tikhonov/datasets/other_exps/rag_retrieve_bench.csv"
)


In [31]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "Qwen/Qwen2.5-14B-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)


config.json:   0%|          | 0.00/663 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/47.5k [00:00<?, ?B/s]

model-00001-of-00008.safetensors:   0%|          | 0.00/3.89G [00:00<?, ?B/s]

model-00002-of-00008.safetensors:   0%|          | 0.00/4.00G [00:00<?, ?B/s]

model-00003-of-00008.safetensors:   0%|          | 0.00/4.00G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

In [33]:
import transformers

def get_response(model: transformers.AutoModelForCausalLM, prompt: str) -> str:
    messages = [
        {"role": "system", "content": "You are Qwen, created by Alibaba Cloud. You are a helpful assistant."},
        {"role": "user", "content": prompt}
    ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)
    
    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=512
    )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]
    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    return response


In [44]:
from tqdm import tqdm

for data in [crypto_retrieve_bench, crypto_qna_benchmark]:
    responses = []
    for idx in tqdm(range(data.shape[0])):
        query = data.queries.iloc[idx]
        answer = data.answers.iloc[idx]
        prompt = f"""Evaluate the relevance of the following answer to the given query.
Output 1 if the answer is relevant and directly addresses the query.
Output 0 if the answer is not relevant or does not address the query. Only output 0 or 1 with no additional explanation.

Query: {query}
Answer: {answer}

Output:

[0 or 1]
"""
        responses.append(get_response(
            model, prompt
        ))
    data["labels"] = responses


100%|██████████| 771/771 [01:40<00:00,  7.64it/s]


In [48]:
crypto_retrieve_bench_ft = crypto_retrieve_bench[crypto_retrieve_bench.labels == "1"].copy()


In [50]:
crypto_qna_benchmark_ft = crypto_qna_benchmark[crypto_qna_benchmark.labels == "1"].copy()


In [51]:
crypto_qna_benchmark_ft.to_csv(
    "/share/a.tikhonov/datasets/other_exps/qna_filtered_benchmark_ft.csv",
    index=False
)
crypto_retrieve_bench_ft.to_csv(
    "/share/a.tikhonov/datasets/other_exps/rag_retrieve_bench_ft.csv",
    index=False
)


# Evaluation & Metrics

In [5]:
import numpy as np

def mean_average_precision(scores, relevance):
    num_queries, num_docs = scores.shape
    average_precisions = []

    for i in range(num_queries):
        query_scores = scores[i]
        query_relevance = relevance[i]
        
        # Sort documents by scores in descending order
        sorted_indices = np.argsort(-query_scores)  # Negative for descending sort
        sorted_relevance = query_relevance[sorted_indices]

        precision_at_k = []
        num_relevant = 0

        for k in range(num_docs):
            if sorted_relevance[k] == 1:
                num_relevant += 1
                precision = num_relevant / (k + 1) 
                precision_at_k.append(precision)
        
        total_relevant = np.sum(query_relevance)
        if total_relevant > 0:
            average_precision = np.sum(precision_at_k) / total_relevant
        else:
            average_precision = 0
        
        average_precisions.append(average_precision)
    
    # Calculate mean average precision
    mean_ap = np.mean(average_precisions)
    return mean_ap



In [6]:
import numpy as np

def mean_reciprocal_rank_top_n(scores, relevance, N):
    num_queries = scores.shape[0]
    reciprocal_ranks = []

    for i in range(num_queries):
        # Sort documents by descending scores for query i
        query_scores = scores[i]
        query_relevance = relevance[i]
        sorted_indices = np.argsort(-query_scores)  # Sort in descending order
        sorted_relevance = query_relevance[sorted_indices]

        # Consider only the top N documents
        sorted_relevance_top_n = sorted_relevance[:N]

        # Find the rank of the first relevant document in top N
        first_relevant_rank = np.where(sorted_relevance_top_n == 1)[0]
        if len(first_relevant_rank) > 0:
            reciprocal_rank = 1 / (first_relevant_rank[0] + 1)  # +1 because ranks start at 1
        else:
            reciprocal_rank = 0  # No relevant documents in top N

        reciprocal_ranks.append(reciprocal_rank)

    # Compute the mean of reciprocal ranks
    mrr = np.mean(reciprocal_ranks)
    return mrr


# DPR models

In [8]:
import pandas as pd

crypto_qna_benchmark_ft = pd.read_csv(
    "/share/a.tikhonov/datasets/other_exps/qna_filtered_benchmark_ft.csv"
)
crypto_retrieve_bench_ft = pd.read_csv(
    "/share/a.tikhonov/datasets/other_exps/rag_retrieve_bench_ft.csv",
)


In [9]:
crypto_qna_benchmark_ft.shape, crypto_retrieve_bench_ft.shape


((215, 3), (239, 3))

In [10]:
import torch
from sentence_transformers import SentenceTransformer

def get_dpr_scores(
    model_id_or_path: str,
    retrieval_data: pd.DataFrame,
    
):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = SentenceTransformer(model_id_or_path, trust_remote_code=True).to(device)
    
    queries_embed = model.encode(retrieval_data["queries"].tolist())
    responses_embed = model.encode(retrieval_data["answers"].tolist())
    sim_scores = model.similarity(queries_embed, responses_embed)
    labels = torch.eye(*sim_scores.shape)
    return {
        "scores": sim_scores.numpy(),
        "relevance": labels.numpy()
    }



/opt/conda/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")


In [11]:
def get_retrieval_metrics_dpr(
    data: pd.DataFrame,
    model_id_or_path: str,
    N: int
):
    dpr_results = get_dpr_scores(
        model_id_or_path,
        data
    )
    map_score = mean_average_precision(**dpr_results)
    mrr_score = mean_reciprocal_rank_top_n(dpr_results["scores"], dpr_results["relevance"], N)
    print(f"Mean Average Precision (mAP): {map_score:.4f}")
    print(f"Mean Reciprocal Rank (MRR): {mrr_score:.4f}")
    return map_score, mrr_score
    

In [15]:
dpr_models_sentence_transformers = [
    'paraphrase-MiniLM-L3-v2',
    'all-MiniLM-L6-v2',
    'multi-qa-MiniLM-L6-cos-v1',
    'paraphrase-multilingual-MiniLM-L12-v2',
    'all-MiniLM-L12-v2',
    'paraphrase-albert-small-v2',
    'distiluse-base-multilingual-cased-v1',
    'distiluse-base-multilingual-cased-v2',
    'all-distilroberta-v1',
    'multi-qa-distilbert-cos-v1',
    'all-mpnet-base-v2',
    'multi-qa-mpnet-base-dot-v1',
    'paraphrase-multilingual-mpnet-base-v2',
    # "dunzhang/stella_en_400M_v5",
    # "jinaai/jina-embeddings-v3",
    # "Alibaba-NLP/gte-large-en-v1.5",
    # "jxm/cde-small-v1",
    # "HIT-TMG/KaLM-embedding-multilingual-mini-instruct-v1",
    # "WhereIsAI/UAE-Large-V1"
]


In [16]:
from tqdm import tqdm
import pandas as pd

dpr_metrics = {
    "model_id": [],
    "bench_name": [],
    "mAP": [],
    "MRR": []
}

benchmarks = {
    "cryptoqa-2024": crypto_qna_benchmark_ft,
    "HemaChandrao/crypto_QA": crypto_retrieve_bench_ft
}

for bench_name, qna_data in benchmarks.items():
    print(f"Current benchmark: {bench_name}")
    for model_id in tqdm(dpr_models_sentence_transformers):
        print(f"\tCurrent model: {model_id}")
        map_score, mrr_score = get_retrieval_metrics_dpr(qna_data, model_id, N=10)
        dpr_metrics["model_id"].append(model_id)
        dpr_metrics["bench_name"].append(bench_name)
        dpr_metrics["mAP"].append(map_score)
        dpr_metrics["MRR"].append(mrr_score)
        

dpr_metrics_df = pd.DataFrame(dpr_metrics)


Current benchmark: cryptoqa-2024


  0%|          | 0/13 [00:00<?, ?it/s]

	Current model: paraphrase-MiniLM-L3-v2


  8%|▊         | 1/13 [00:02<00:28,  2.39s/it]

Mean Average Precision (mAP): 0.2564
Mean Reciprocal Rank (MRR): 0.2407
	Current model: all-MiniLM-L6-v2


...

Mean Average Precision (mAP): 0.7983
Mean Reciprocal Rank (MRR): 0.7969
	Current model: multi-qa-mpnet-base-dot-v1


 92%|█████████▏| 12/13 [00:33<00:02,  2.85s/it]

Mean Average Precision (mAP): 0.7789
Mean Reciprocal Rank (MRR): 0.7770
	Current model: paraphrase-multilingual-mpnet-base-v2


100%|██████████| 13/13 [00:38<00:00,  2.96s/it]

Mean Average Precision (mAP): 0.6102
Mean Reciprocal Rank (MRR): 0.6018


In [17]:
dpr_metrics_df.sort_values(["mAP", "MRR"], ascending=[False, False])


,model_id,bench_name,mAP,MRR
23,all-mpnet-base-v2,HemaChandrao/crypto_QA,0.798267,0.796895
24,multi-qa-mpnet-base-dot-v1,HemaChandrao/crypto_QA,0.778915,0.777039
21,all-distilroberta-v1,HemaChandrao/crypto_QA,0.733526,0.730228
17,all-MiniLM-L12-v2,HemaChandrao/crypto_QA,0.725623,0.722476
15,multi-qa-MiniLM-L6-cos-v1,HemaChandrao/crypto_QA,0.716872,0.714196
22,multi-qa-distilbert-cos-v1,HemaChandrao/crypto_QA,0.713394,0.710679
14,all-MiniLM-L6-v2,HemaChandrao/crypto_QA,0.712731,0.709069
25,paraphrase-multilingual-mpnet-base-v2,HemaChandrao/crypto_QA,0.610216,0.601813
18,paraphrase-albert-small-v2,HemaChandrao/crypto_QA,0.607011,0.601449
16,paraphrase-multilingual-MiniLM-L12-v2,HemaChandrao/crypto_QA,0.594264,0.585709


In [14]:
dpr_metrics_df.sort_values(["mAP", "MRR"], ascending=[False, False])


,model_id,bench_name,mAP,MRR
8,HIT-TMG/KaLM-embedding-multilingual-mini-instr...,HemaChandrao/crypto_QA,0.818385,0.817797
5,jinaai/jina-embeddings-v3,HemaChandrao/crypto_QA,0.807477,0.805951
6,Alibaba-NLP/gte-large-en-v1.5,HemaChandrao/crypto_QA,0.779034,0.777258
9,WhereIsAI/UAE-Large-V1,HemaChandrao/crypto_QA,0.754700,0.752776
1,Alibaba-NLP/gte-large-en-v1.5,cryptoqa-2024,0.631214,0.623856
3,HIT-TMG/KaLM-embedding-multilingual-mini-instr...,cryptoqa-2024,0.608928,0.602966
0,jinaai/jina-embeddings-v3,cryptoqa-2024,0.608519,0.601709
4,WhereIsAI/UAE-Large-V1,cryptoqa-2024,0.554994,0.547885
7,jxm/cde-small-v1,HemaChandrao/crypto_QA,0.240464,0.224688
2,jxm/cde-small-v1,cryptoqa-2024,0.155341,0.136702


# CrossEncoders

In [ ]:
from sentence_transformers.cross_encoder import CrossEncoder

model = CrossEncoder("cross-encoder/stsb-distilroberta-base")

sentence_combinations = [[query, sentence] for sentence in corpus]
# scores = model.predict(sentence_combinations)
